<a href="https://colab.research.google.com/github/giuliacern/HyperfMNIST/blob/master/Iris_Optimization2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install GPyOpt
!pip uninstall tensorflow
!pip install tensorflow==2.0.0
import tensorflow as tf
print(tf.__version__)

In [0]:
import GPy, GPyOpt
import numpy as np
import pandas as pd
import random

from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder

In [0]:
# IRIS class
class IRIS():
    def __init__(self, l1, first_input=4, last_output=3,
                 batch_size=10, 
                 epochs=10, 
                 validation_split=0.1):
        self.__l1 = l1
        self.__first_input = first_input
        self.__last_output = last_output
        self.batch_size = batch_size
        self.epochs = epochs
        self.validation_split = validation_split
        self.__x_train, self.__x_test, self.__y_train, self.__y_test = self.iris_data()
        self.__model = self.iris_model()
        
    def iris_data(self):
        iris_data = load_iris() # load the iris dataset
        x = iris_data.data
        y_ = iris_data.target.reshape(-1, 1) # Convert data to a single column
        # One Hot encode the class labels
        encoder = OneHotEncoder(sparse=False)
        y = encoder.fit_transform(y_)
        # Split the data for training and testing
        X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.20)
        return X_train, X_test, Y_train, Y_test
    
    
    
    def iris_model(self):
        model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(self.__l1, input_shape=(self.__first_input,), activation=tf.nn.relu),
        tf.keras.layers.Dense(self.__last_output, activation=tf.nn.softmax),
        ])
        
        model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
        return model
    
    def iris_fit(self):        
        self.__model.fit(self.__x_train, self.__y_train,
                       batch_size=self.batch_size,
                       epochs=self.epochs,
                       verbose=0,
                       validation_split=self.validation_split)
    
    def iris_evaluate(self):
        self.iris_fit()   
        evaluation = self.__model.evaluate(self.__x_test, self.__y_test, batch_size=self.batch_size, verbose=0)
        return evaluation

In [0]:
def run_iris(l1=512, first_input=4, last_output=3,
              batch_size=10, epochs=1, validation_split=0.1):
    
    _iris = IRIS(l1 = l1, first_input=first_input, last_output=last_output,
                   batch_size=batch_size, epochs=epochs, 
                   validation_split=validation_split)
    iris_evaluation = _iris.iris_evaluate()
    return iris_evaluation

In [7]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# the bounds dict should be in order of continuous type and then discrete type
domain = [{'name': 'validation_split', 'type': 'continuous',  'domain': (0.1, 0.3)},
          {'name': 'batch_size',       'type': 'discrete',    'domain': (10, 50)},
          {'name': 'epochs',           'type': 'discrete',    'domain': (5, 10, 20)},
          {'name': 'l1',               'type': 'discrete',    'domain': (64, 128, 256, 512, 1024)}
]

def f(x):
    print("values on domain: ", x)
    evaluation = run_iris(
        batch_size = int(x[:,1]), 
        epochs = int(x[:,2]), 
        l1 = int(x[:,3]),
        validation_split = float(x[:,0]))
    print("LOSS:\t{0} \t ACCURACY:\t{1}".format(evaluation[0], evaluation[1]))
    print(evaluation)
    print("\n")
    return evaluation[0]

# optimizer
opt_iris = GPyOpt.methods.BayesianOptimization(f=f, domain=domain)
opt_iris.run_optimization(max_iter=10)
        
opt_iris.plot_acquisition()

values on domain:  [[ 0.20189694 50.         10.         64.        ]]
LOSS:	1.0608206987380981 	 ACCURACY:	0.2666666805744171
[1.0608206987380981, 0.26666668]


values on domain:  [[1.06203632e-01 5.00000000e+01 2.00000000e+01 1.28000000e+02]]
LOSS:	0.6568496227264404 	 ACCURACY:	1.0
[0.6568496227264404, 1.0]


values on domain:  [[2.44076293e-01 1.00000000e+01 1.00000000e+01 2.56000000e+02]]
LOSS:	0.44111714760462445 	 ACCURACY:	0.9666666388511658
[0.44111714760462445, 0.96666664]


values on domain:  [[2.21986014e-01 5.00000000e+01 5.00000000e+00 2.56000000e+02]]
LOSS:	0.9036325812339783 	 ACCURACY:	0.8999999761581421
[0.9036325812339783, 0.9]


values on domain:  [[1.08454972e-01 5.00000000e+01 5.00000000e+00 1.02400000e+03]]
LOSS:	0.6652805209159851 	 ACCURACY:	0.7333333492279053
[0.6652805209159851, 0.73333335]


values on domain:  [[1.00e-01 1.00e+01 1.00e+01 2.56e+02]]
LOSS:	0.40580328305562335 	 ACCURACY:	0.8333333134651184
[0.40580328305562335, 0.8333333]


values on domain: 

In [8]:
print("""
Optimized Parameters:
\t{0}:\t{1}
\t{2}:\t{3}
\t{4}:\t{5}
\t{6}:\t{7}
""".format(domain[0]["name"], opt_iris.x_opt[0],
           domain[1]["name"], opt_iris.x_opt[1],
           domain[2]["name"], opt_iris.x_opt[2],
           domain[3]["name"], opt_iris.x_opt[3]))


Optimized Parameters:
	validation_split:	0.1
	batch_size:	10.0
	epochs:	10.0
	l1:	256.0



In [9]:
print("optimized loss: {0}".format(opt_iris.fx_opt))

optimized loss: 0.40580328305562335


In [0]:
from sklearn import datasets
from sklearn import tree        #importing subpackage from sklearn library
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
iris = datasets.load_iris()
from sklearn.model_selection import GridSearchCV


In [10]:
# optimizer
opt_iris = GPyOpt.methods.BayesianOptimization(f=f, domain=domain, acquisition_type ='LCB', verbosity = True, verbosity_model= True)
opt_iris.run_optimization(max_iter=10)
        
opt_iris.plot_acquisition()

values on domain:  [[ 0.29415667 50.          5.         64.        ]]
LOSS:	1.1540392637252808 	 ACCURACY:	0.36666667461395264
[1.1540392637252808, 0.36666667]


values on domain:  [[1.1185428e-01 5.0000000e+01 2.0000000e+01 1.2800000e+02]]
LOSS:	0.6667775511741638 	 ACCURACY:	0.6000000238418579
[0.6667775511741638, 0.6]


values on domain:  [[ 0.16407906 50.         20.         64.        ]]
LOSS:	0.8384746313095093 	 ACCURACY:	0.7333333492279053
[0.8384746313095093, 0.73333335]


values on domain:  [[  0.15866471  10.           5.         128.        ]]
LOSS:	0.688673754533132 	 ACCURACY:	0.6666666865348816
[0.688673754533132, 0.6666667]


values on domain:  [[2.67396816e-01 5.00000000e+01 5.00000000e+00 1.02400000e+03]]
LOSS:	0.7415908575057983 	 ACCURACY:	0.6000000238418579
[0.7415908575057983, 0.6]


values on domain:  [[  0.16197367  10.           5.         128.        ]]
LOSS:	0.5929286678632101 	 ACCURACY:	0.6666666865348816
[0.5929286678632101, 0.6666667]


values on domain:

In [11]:
print("""
Optimized Parameters:
\t{0}:\t{1}
\t{2}:\t{3}
\t{4}:\t{5}
\t{6}:\t{7}
""".format(domain[0]["name"], opt_iris.x_opt[0],
           domain[1]["name"], opt_iris.x_opt[1],
           domain[2]["name"], opt_iris.x_opt[2],
           domain[3]["name"], opt_iris.x_opt[3]))


Optimized Parameters:
	validation_split:	0.3
	batch_size:	10.0
	epochs:	20.0
	l1:	256.0



In [12]:
print("optimized loss: {0}".format(opt_iris.fx_opt))

optimized loss: 0.282694935798645


In [13]:
# Monte Carlo Markov Chain 
opt_iris = GPyOpt.methods.BayesianOptimization(model_type = 'GP_MCMC', f=f, domain=domain, acquisition_type ='EI_MCMC', verbosity = True, verbosity_model= True)
opt_iris.run_optimization(max_iter=10)
opt_iris.plot_acquisition()


print("""
Optimized Parameters:
\t{0}:\t{1}
\t{2}:\t{3}
\t{4}:\t{5}
\t{6}:\t{7}
""".format(domain[0]["name"], opt_iris.x_opt[0],
           domain[1]["name"], opt_iris.x_opt[1],
           domain[2]["name"], opt_iris.x_opt[2],
           domain[3]["name"], opt_iris.x_opt[3]))

values on domain:  [[1.86353386e-01 1.00000000e+01 5.00000000e+00 1.02400000e+03]]
LOSS:	0.46791069706281024 	 ACCURACY:	0.9333333373069763
[0.46791069706281024, 0.93333334]


values on domain:  [[  0.26721114  50.          10.         256.        ]]
LOSS:	0.7337373495101929 	 ACCURACY:	0.800000011920929
[0.7337373495101929, 0.8]


values on domain:  [[1.667434e-01 1.000000e+01 1.000000e+01 2.560000e+02]]
LOSS:	0.5054951707522074 	 ACCURACY:	0.8666666746139526
[0.5054951707522074, 0.8666667]


values on domain:  [[ 0.12847475 10.          5.         64.        ]]
LOSS:	0.7371299266815186 	 ACCURACY:	0.6000000238418579
[0.7371299266815186, 0.6]


values on domain:  [[  0.29292095  10.           5.         128.        ]]


reconstraining parameters GP_regression.rbf
reconstraining parameters GP_regression.Gaussian_noise.variance


LOSS:	0.6711524526278178 	 ACCURACY:	0.699999988079071
[0.6711524526278178, 0.7]


values on domain:  [[1.86353644e-01 1.00000000e+01 5.00000000e+00 1.02400000e+03]]
LOSS:	0.3614371716976166 	 ACCURACY:	1.0
[0.3614371716976166, 1.0]


values on domain:  [[1.86356514e-01 1.00000000e+01 5.00000000e+00 1.02400000e+03]]
LOSS:	0.49149157603581745 	 ACCURACY:	0.8999999761581421
[0.49149157603581745, 0.9]


values on domain:  [[1.45768571e-01 1.00000000e+01 5.00000000e+00 1.02400000e+03]]
LOSS:	0.4054810404777527 	 ACCURACY:	0.9666666388511658
[0.4054810404777527, 0.96666664]


values on domain:  [[1.000e-01 1.000e+01 5.000e+00 1.024e+03]]
LOSS:	0.43198809027671814 	 ACCURACY:	0.699999988079071
[0.43198809027671814, 0.7]


values on domain:  [[1.35498632e-01 1.00000000e+01 5.00000000e+00 1.02400000e+03]]
LOSS:	0.4013886551062266 	 ACCURACY:	1.0
[0.4013886551062266, 1.0]


values on domain:  [[1.000e-01 1.000e+01 5.000e+00 1.024e+03]]
LOSS:	0.43741175532341003 	 ACCURACY:	0.8666666746139526
[0

In [0]:
from sklearn.neural_network import MLPClassifier

parameter_space = {
    'hidden_layer_sizes': [(64,), (128,), (256,), (512,), (1024,)],
    'batch_size':[10, 50],
    'max_iter': [5, 10, 20]
}

In [0]:
from sklearn.model_selection import GridSearchCV
x = iris.data
y_ = iris.target.reshape(-1, 1) # Convert data to a single column
# One Hot encode the class labels
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y_)
# Split the data for training and testing
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.20)

In [0]:
mlp = MLPClassifier(solver = 'adam', validation_fraction = 0.1)
clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
clf.fit(X_train, Y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(100,),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_fun=15000,
                                     max_iter=200, momentum=0.9,
                                     n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_state=None, shuffle=True,
                                     solver='adam', tol=0.0001,
                                     validation_fraction=0.1, verbose=False,
                                     warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'

In [0]:
# Best paramete set
print('Best parameters found:\n', clf.best_params_)

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))


Best parameters found:
 {'batch_size': 10, 'hidden_layer_sizes': (1024,), 'max_iter': 20}
0.308 (+/-0.201) for {'batch_size': 10, 'hidden_layer_sizes': (64,), 'max_iter': 5}
0.483 (+/-0.366) for {'batch_size': 10, 'hidden_layer_sizes': (64,), 'max_iter': 10}
0.683 (+/-0.047) for {'batch_size': 10, 'hidden_layer_sizes': (64,), 'max_iter': 20}
0.417 (+/-0.232) for {'batch_size': 10, 'hidden_layer_sizes': (128,), 'max_iter': 5}
0.658 (+/-0.062) for {'batch_size': 10, 'hidden_layer_sizes': (128,), 'max_iter': 10}
0.683 (+/-0.047) for {'batch_size': 10, 'hidden_layer_sizes': (128,), 'max_iter': 20}
0.433 (+/-0.309) for {'batch_size': 10, 'hidden_layer_sizes': (256,), 'max_iter': 5}
0.683 (+/-0.047) for {'batch_size': 10, 'hidden_layer_sizes': (256,), 'max_iter': 10}
0.683 (+/-0.047) for {'batch_size': 10, 'hidden_layer_sizes': (256,), 'max_iter': 20}
0.658 (+/-0.062) for {'batch_size': 10, 'hidden_layer_sizes': (512,), 'max_iter': 5}
0.683 (+/-0.047) for {'batch_size': 10, 'hidden_layer_siz

In [0]:
y_true, y_pred = Y_test , clf.predict(X_test)

from sklearn.metrics import classification_report
print('Results on the test set:')
print(classification_report(y_true, y_pred))


Results on the test set:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.92      0.96        12
           2       1.00      0.75      0.86         8

   micro avg       1.00      0.90      0.95        30
   macro avg       1.00      0.89      0.94        30
weighted avg       1.00      0.90      0.94        30
 samples avg       0.90      0.90      0.90        30



In [0]:
print("Accuracy of the tuned model: %.4f" %clf.best_score_)
print(clf.best_params_)

Accuracy of the tuned model: 0.7500
{'batch_size': 10, 'hidden_layer_sizes': (1024,), 'max_iter': 20}


In [0]:
clf.score(X_train, Y_train)

0.9333333333333333

In [0]:
from sklearn.model_selection import RandomizedSearchCV

In [0]:
random = RandomizedSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
random.fit(X_train, Y_train)

RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                           batch_size='auto', beta_1=0.9,
                                           beta_2=0.999, early_stopping=False,
                                           epsilon=1e-08,
                                           hidden_layer_sizes=(100,),
                                           learning_rate='constant',
                                           learning_rate_init=0.001,
                                           max_fun=15000, max_iter=200,
                                           momentum=0.9, n_iter_no_change=10,
                                           nesterovs_momentum=True, power_t=0.5,
                                           random...fle=True,
                                           solver='adam', tol=0.0001,
                                           validation_fraction=0.1,
                                

In [0]:
# Best paramete set
print('Best parameters found:\n', random.best_params_)

# All results
means = random.cv_results_['mean_test_score']
stds = random.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, random.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

Best parameters found:
 {'max_iter': 20, 'hidden_layer_sizes': (1024,), 'batch_size': 10}
0.108 (+/-0.306) for {'max_iter': 5, 'hidden_layer_sizes': (1024,), 'batch_size': 50}
0.508 (+/-0.295) for {'max_iter': 5, 'hidden_layer_sizes': (256,), 'batch_size': 10}
0.108 (+/-0.306) for {'max_iter': 20, 'hidden_layer_sizes': (64,), 'batch_size': 50}
0.667 (+/-0.047) for {'max_iter': 20, 'hidden_layer_sizes': (256,), 'batch_size': 50}
0.667 (+/-0.047) for {'max_iter': 10, 'hidden_layer_sizes': (128,), 'batch_size': 10}
0.708 (+/-0.085) for {'max_iter': 20, 'hidden_layer_sizes': (1024,), 'batch_size': 10}
0.117 (+/-0.262) for {'max_iter': 20, 'hidden_layer_sizes': (128,), 'batch_size': 50}
0.458 (+/-0.366) for {'max_iter': 10, 'hidden_layer_sizes': (512,), 'batch_size': 50}
0.108 (+/-0.306) for {'max_iter': 5, 'hidden_layer_sizes': (256,), 'batch_size': 50}
0.125 (+/-0.147) for {'max_iter': 5, 'hidden_layer_sizes': (64,), 'batch_size': 10}


In [0]:
y_true, y_pred = Y_test , random.predict(X_test)

from sklearn.metrics import classification_report
print('Results on the test set:')
print(classification_report(y_true, y_pred))

Results on the test set:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.83      0.91        12
           2       1.00      1.00      1.00         8

   micro avg       1.00      0.93      0.97        30
   macro avg       1.00      0.94      0.97        30
weighted avg       1.00      0.93      0.96        30
 samples avg       0.93      0.93      0.93        30



In [0]:
random.score(X_train, Y_train)

0.9166666666666666